# Notebooks

> implements the Jupyter Notebook Module Finder. See [Importing Jupyter Notebooks as Modules](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html) for more details.

In [ ]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export
import logging, inspect, itertools
from pathlib import Path

from dataclasses import dataclass, field, KW_ONLY
from typing import Union, Optional, List

from urllib3.util.url import parse_url
from iza.utils import (
    stream_file, Directory, urljoin,
    is_tarball, is_gz, decompress_tarball, decompress_gunzip
)

In [2]:
from iza.utils import ArchiveDownloader, AmazonArchiveDownloader
from iza.static import AMAZON_BUCKET_FLUENTBIO

In [3]:
downloader = ArchiveDownloader(
    rootdir = AMAZON_BUCKET_FLUENTBIO,
    archive = 'public-datasets/pbmc/',
    entries = ['combined.html', 'filtered_matrix.tar.gz'],
    savedir = '~/Downloads/TEST',  extract=True, cleanup=True
)
downloader.execute()  # Will download and extract all the entries

Processing archive: public-datasets/pbmc/

Output()

]8;id=273256;file:///Users/solst/Downloads/TEST\/Users/solst/Downloads/TEST]8;;\
┣━━ ]8;id=968283;file:///Users/solst/Downloads/TEST/filtered_matrix\filtered_matrix]8;;\ (224 bytes)
┃   ┣━━ ]8;id=516437;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_1\sensitivity_1]8;;\ (160 bytes)
┃   ┃   ┣━━ ]8;id=360999;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_1/barcodes.tsv.gz\barcodes.tsv.gz]8;;\ (3.8 kB)
┃   ┃   ┣━━ ]8;id=982817;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_1/features.tsv.gz\features.tsv.gz]8;;\ (288.2 kB)
┃   ┃   ┗━━ ]8;id=166422;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_1/matrix.mtx.gz\matrix.mtx.gz]8;;\ (4.4 MB)
┃   ┣━━ ]8;id=773225;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_2\sensitivity_2]8;;\ (160 bytes)
┃   ┃   ┣━━ ]8;id=458227;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_2/barcodes.tsv.gz\barcodes.tsv.gz]8;;\ (24.4 kB)
┃   ┃   ┣━━ ]8;id=878644;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_2/features.tsv.gz\features.tsv.gz]8;;\ (288.2 kB)
┃   ┃   ┗━━ ]8;id=507929;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_2/matrix.mtx.gz\matrix.mtx.gz]8;;\ (21.8 MB)
┃   ┣━━ ]8;id=707774;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_3\sensitivity_3]8;;\ (160 bytes)
┃   ┃   ┣━━ ]8;id=401636;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_3/barcodes.tsv.gz\barcodes.tsv.gz]8;;\ (65.8 kB)
┃   ┃   ┣━━ ]8;id=730509;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_3/features.tsv.gz\features.tsv.gz]8;;\ (288.2 kB)
┃   ┃   ┗━━ ]8;id=435490;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_3/matrix.mtx.gz\matrix.mtx.gz]8;;\ (48.9 MB)
┃   ┣━━ ]8;id=139307;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_4\sensitivity_4]8;;\ (160 bytes)
┃   ┃   ┣━━ ]8;id=902198;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_4/barcodes.tsv.gz\barcodes.tsv.gz]8;;\ (97.1 kB)
┃   ┃   ┣━━ ]8;id=760527;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_4/features.tsv.gz\features.tsv.gz]8;;\ (288.2 kB)
┃   ┃   ┗━━ ]8;id=639770;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_4/matrix.mtx.gz\matrix.mtx.gz]8;;\ (65.4 MB)
┃   ┗━━ ]8;id=464190;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_5\sensitivity_5]8;;\ (160 bytes)
┃       ┣━━ ]8;id=339069;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_5/barcodes.tsv.gz\barcodes.tsv.gz]8;;\ (116.6 kB)
┃       ┣━━ ]8;id=664470;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_5/features.tsv.gz\features.tsv.gz]8;;\ (288.2 kB)
┃       ┗━━ ]8;id=310455;file:///Users/solst/Downloads/TEST/filtered_matrix/sensitivity_5/matrix.mtx.gz\matrix.mtx.gz]8;;\ (72.9 MB)
┣━━ ]8;id=936094;file:///Users/solst/Downloads/TEST/combined.html\combined.html]8;;\ (30.6 MB)
┗━━ ]8;id=817307;file:///Users/solst/Downloads/TEST/filtered_matrix.tar.gz\filtered_matrix.tar.gz]8;;\ (215.2 MB)

### Typer

In [ ]:
#| export
#| eval: False
# import typer
# app = typer.Typer()
# @app.command()
# def download(rootdir: str, archive: str, entries: List[str], savedir: str, extract: bool = False, cleanup: bool = False):
#     downloader = ArchiveDownloader(rootdir, archive, entries, savedir, extract, cleanup)
#     downloader.execute()

# @app.command()
# def download_from_amazon(bucket: str, archive: str, entries: List[str], savedir: str, extract: bool = False, cleanup: bool = False, region: str = 'us-east-2'):
#     downloader = AmazonArchiveDownloader(bucket, archive, entries, savedir, extract, cleanup, region)
#     downloader.execute()


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()